# 🧠 Iterators, Function Copying, Closures & Decorators, in Python - Explained Like You're Chatting Over Coffee

If you've ever found yourself scratching your head while trying to wrap your brain around iterators, closures, decorators, or why Python lets you pass functions around like trading cards, you're not alone.
These concepts are cornerstones of Python's flexibility. They can seem mysterious at first, but once you get them, you'll feel like you've unlocked a new superpower.
In this article, we'll demystify:

- ✅ What closures are?
- ✅ How function copying works!!!
- ✅ How decorators work under the hood!!!

## 📦 First up: Functions are first-class citizens in Python

In Python, functions are first-class citizens, meaning they can be assigned to variables, passed as arguments, and even returned from other functions.
- Yep, just like strings, integers, or lists, if we look at the code below.

In [2]:
def greet(name):
 return f'Hello, {name}!'

say_hello = greet  
'''
 - Assigning the function name 'greet' to a variable 'say_hello'
 - Note no parentheses () at the end of the function name
'''

print(say_hello('Anshu'))

Hello, Anshu!


What just happened here? We actually didn't "**copy**" the function, but rather **created another reference** to the same object.

In [3]:
print(greet is say_hello)       # True

True


Python functions are objects. Assigning a function to another variable doesn't duplicate it - it just references the same object in memory.

#### 📋 When Would You Want to Copy a Function?
- You want to modify a copy without affecting the original (e.g., for monkey-patching or decorators).
- You need to clone behavior across modules or test environments.
- You're dynamically generating or modifying functions.

#### Types Copy Example
If you really need to copy a function:

In [21]:
import types

def greet(name):
    return f"Hi {name}"

clone = types.FunctionType(
    greet.__code__, 
    greet.__globals__, 
    name=greet.__name__, 
    argdefs=greet.__defaults__, 
    closure=greet.__closure__
)

print(clone("Anshu"))  # Hi Anshu
print(f'{clone is greet = }')   # clone is greet = False

Hi Anshu
clone is greet = False


#### 🧠 What's happening here?

| Parameter | What it Represents |
|:---------:|:------------------:|
| _ _ code_ _ | The compiled bytecode |
| _ _ globals_ _ |	Global variables accessible to the function |
| _ _ name_ _ | Name of the function |
| _ _ defaults_ _ | Default arguments |
| _ _ closure_ _ | For closures (inner function referencing outer vars) |

#### 🏁 Summary
- Functions in Python are objects - copying them means dealing with references.
- You can use types.FunctionType to create a shallow copy.
- Full deep copies are rare and usually unnecessary.
- When in doubt, prefer clean redefinition or wrapping over copying.

----

# 🔐 Closures: Functions with Memory

Imagine you\'re at your favorite coffee shop. You order your usual - an Espresso Macchiato. The barista remembers your order and smiles: "The usual?" That memory is exactly what a closure is.

A **closure** is a function object that "remembers" the variables/values from the enclosing scope where it was created, even after that scope has finished execution / is no longer available.

#### Core Concepts of Closures
- Nested Function: A closure always involves a function defined inside another function.
- Free Variables: The inner function refers to variables from the outer function.
- Returning Functions: The outer function returns the inner function.
- State Preservation: The returned function "remembers" the environment in which it was created.

In [5]:
def make_multiplier(factor):    # Outer Function
    def multiply(number):       # Inner Function 
        return number * factor  # 'factor' from outer function is remembered
    return multiply             # Inner function being returned 

double = make_multiplier(2)     # double is multiply function with factor==>2
triple = make_multiplier(3)     # triple is multiply function with factor==>3

print(double(5))  # 10          # returns 10 as number==>5 * factor==> 2 = 10
print(triple(5))  # 15          # returns 15 as number==>5 * factor==> 3 = 15

10
15


The key concept here is a **closure**. Even though the `make_multiplier` function has *finished executing*, the returned `multiply function` which still remembers the value of factor from its enclosing scope, that was active at the time of creation, even after the outer function has finished executing.

So `double` remembers `factor = 2`, and `triple` remembers `factor = 3`.

That's a *closure*. A function with an inner function that captures variables from the outer function.

Still, want proof that `double` actually `closed over` a variable?

In [6]:
print(double.__closure__[0].cell_contents)  # 2

2


#### Why Closures Are Useful
- Data Encapsulation: Closures provide a way to hide data from the global scope while still making it accessible to specific functions.
- Creating Function Factories: As shown in your original example, closures enable creating customized functions.
- Implementing Decorators: Closures are fundamental to Python decorators (More on this in next section).

# 🎁 Decorators: Fancy Closures That Wrap Functions

Now that we know what closures are, decorators will feel like a natural extension. **`Decorators`** are a powerful feature in Python that allow you to modify the behavior of functions or classes without permanently changing their source code. Let's explore them in detail.

#### Core Concept
A decorator is a function that takes another function as an argument, adds some functionality, and returns another function. All of this without altering the source code of the original function.
Think of it like putting a present inside a gift box. The gift (original function) is still there - but now it's wrapped in something new.

#### Basic Structure

In [7]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        # Code to be executed before the original function
        result = original_function(*args, **kwargs)
        # Code to be executed after the original function
        return result
    return wrapper_function

#### Simple Example
Here's a basic example of using decorator, that logs when a function is called. Here we use @log_function_call to decorate the greet function to modify the functionality .

In [8]:
def log_function_call(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function: {func.__name__}")
        result = func(*args, **kwargs)
        print(f"Function {func.__name__} completed")
        return result
    return wrapper

@log_function_call
def greet(name):
    print(f"Hello, {name}!")

greet("Anshu")

Calling function: greet
Hello, Anshu!
Function greet completed


#### The @ Syntax
The **`@decorator_name`** syntax is just syntactic sugar. The below two code snippets are equivalent:

In [29]:
# Using @ syntax
@log_function_call
def greet(name):
    print(f"Hello, {name}!")

greet('Anshu')
print('----------------')

# Without @ syntax (manual decoration)
def greet(name):
    print(f"Hello, {name}!")
greet = log_function_call(greet)
greet('Anshu')

Calling function: greet
Hello, Anshu!
Function greet completed
----------------
Calling function: greet
Hello, Anshu!
Function greet completed


#### Decorators with Arguments
You can also create decorators that accept arguments:

In [10]:
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            results = []
            for _ in range(num_times):
                results.append(func(*args, **kwargs))
            return results
        return wrapper
    return decorator_repeat

@repeat(3)
def say_hello(name):
    return f"Hello {name}"

print(say_hello("World"))

['Hello World', 'Hello World', 'Hello World']


#### Stacking Decorators
You can use multiple decorators for a single function. They are applied from bottom to top:

In [11]:
def bold(func):
    def wrapper(*args, **kwargs):
        return f"<b>{func(*args, **kwargs)}</b>"
    return wrapper

def italic(func):
    def wrapper(*args, **kwargs):
        return f"<i>{func(*args, **kwargs)}</i>"
    return wrapper

@bold
@italic
def format_text(text):
    return text

print(format_text("Hello"))  # <b><i>Hello</i></b>

<b><i>Hello</i></b>


#### Preserving Function Metadata
When using decorators, metadata of the original function (like name, docstring) gets lost:

In [12]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def example():
    """This is the docstring"""
    pass

print(example.__name__)  # Outputs: wrapper (not example)
print(example.__doc__)   # Outputs: None (not the docstring)

wrapper
None


You can preserve metadata using the functools.wraps decorator:

In [13]:
from functools import wraps

def my_decorator(func):
    @wraps(func)  # This preserves metadata
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def example():
    """This is the docstring"""
    pass

print(example.__name__)  # Outputs: example
print(example.__doc__)   # Outputs: This is the docstring

example
This is the docstring


### Practical Use Cases
1. Timing Functions:

In [14]:
import time
from functools import wraps

def measure_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.2f} seconds")
        return result
    return wrapper

@measure_time
def slow_function():
    time.sleep(1)
    return "Done"

slow_function()

slow_function took 1.00 seconds


'Done'

2. Caching Return Values:

In [15]:
import time
from functools import wraps

def measure_time(func):
    call_depth = 0  # shared state

    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal call_depth
        is_outermost_call = call_depth == 0

        if is_outermost_call:
            start = time.time()

        call_depth += 1
        result = func(*args, **kwargs)
        call_depth -= 1

        if is_outermost_call:
            end = time.time()
            print(f"[Timing] {func.__name__}({args}) took {end - start:.6f} seconds")

        return result

    return wrapper
    
def memoize(func):
    cache = {}
    @wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

In [16]:
@measure_time
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(35))

[Timing] fibonacci((35,)) took 12.750435 seconds
9227465


In [17]:
@measure_time
@memoize
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(35))  # Only one timing line!
print(fibonacci(35))  # Instant return from cache, no timing printed

[Timing] fibonacci((35,)) took 0.001004 seconds
9227465
[Timing] fibonacci((35,)) took 0.000000 seconds
9227465


## Built-in Decorators
### Python has several built-in decorators:
- @property: Transforms a method into a getter for a property
- @classmethod: Converts a method to a class method
- @staticmethod: Converts a method to a static method
- @abstractmethod: Indicates that the method must be implemented by subclasses

Decorators offer a clean and reusable way to modify or enhance the behavior of functions and classes, making your code more modular and following the DRY (Don't Repeat Yourself) principle.

## 🙌 Final Thoughts

###We've now covered:
✅ Function Copying (deep dive into internals)
✅ Closures (functions with memory)
✅ Decorators (closures in a fancy outfit)

And we did it with real-world metaphors, detailed code, and just the right amount of nerdy charm.
Got feedback? Want to go deeper into functools, contextlib, generators, classes? Drop a comment - I'm always up for more Python conversations!

- Anshu